In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://claudia-mlops/1', creation_time=1718497152443, experiment_id='1', last_update_time=1718497152443, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")  # 'data/green_tripdata_2021-01.parquet')
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")  # 'data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [6]:
from mlflow.tracking import MlflowClient
import os

In [7]:
%set_env RUN_ID=da8a85f0cfc94022891e42f77ed37298

env: RUN_ID=da8a85f0cfc94022891e42f77ed37298


In [8]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = os.environ['RUN_ID']

RUN_ID

'da8a85f0cfc94022891e42f77ed37298'

In [19]:
# client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [21]:
# path = client.download_artifacts(run_id=RUN_ID, path='model')


In [10]:
logged_model = f"s3://claudia-mlops/1/{RUN_ID}/artifacts/model"

In [11]:
model = mlflow.pyfunc.load_model(logged_model)

In [9]:
# path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()

In [24]:
!aws s3 ls s3://


2024-06-15 23:11:33 claudia-mlops
